In [1]:
import numpy as np 
import pandas as pd 

In [2]:
from __future__ import print_function 
from __future__ import division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import time
import os
import copy

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.9.1
Torchvision Version:  0.10.1


In [4]:
# Chọn một trong các model [resnet, alexnet, vgg16]
model_name = "resnet"

# Số các classes trong tập dữ liệu
num_classes = 8

# Số các feature map trong mỗi một class
num_maps = 8

# Batch size 
batch_size = 10

# Số các epochs để huấn luyện dữ liệu 
num_epochs = 20

feature_extract = False

model_savepath = 'models/'
# Checkpoint
'''checkpoint = torch.load('../input/fres101bat10num8d3/fres101bat10num8d3.pth') # checkpoint for continue training'''

"checkpoint = torch.load('../input/fres101bat10num8d3/fres101bat10num8d3.pth') # checkpoint for continue training"

In [5]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Mỗi epoch có hai giai đoạn train và test
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0
            running_corrects_dec = 0

            # Lặp qua data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                
                optimizer.zero_grad()

                
                with torch.set_grad_enabled(phase == 'train'):
                    
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = 0.5*loss1 + 0.5*loss2
                    else:
                        outputs1, outputs2 = model(inputs)
                        loss1 = criterion(outputs1, labels)
                        loss2 = criterion(outputs2, labels)
                        loss = 0.5*loss1 + 0.5*loss2

                    _, preds = torch.max(outputs2, 1)
                    _, preds_dec = torch.max(outputs1, 1)

                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        lr = optimizer.param_groups[0]['lr']

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_corrects_dec += torch.sum(preds_dec == labels.data)
                
            if phase == 'train':
                scheduler_ft.step() 
                torch.save({
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer_ft.state_dict(),
                            'epoch': epoch + 1,
                            }, '/models' + str(epoch + 1) + '.pt')
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_acc_dec = running_corrects_dec.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f} Dec: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_acc_dec))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
            if phase == 'test':
                
                val_acc_history.append(epoch_acc)
            
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

class ResNetWSL(nn.Module):
    
    def __init__(self, model, num_classes, num_maps, pooling, pooling2):
        super(ResNetWSL, self).__init__()
        print(num_classes)
        print(num_maps)
        self.features = nn.Sequential(*list(model.children())[:-2])
        # self.num_ftrs = model.fc.in_features

        self.downconv = nn.Sequential(
            nn.Conv2d(512, num_classes*num_maps, kernel_size=1, stride=1, padding=0, bias=True))
        
        self.GAP = nn.AvgPool2d(14)
        self.GMP = nn.MaxPool2d(14)
        self.spatial_pooling = pooling
        self.spatial_pooling2 = pooling2
        self.classifier = nn.Sequential(
            nn.Linear(1024, num_classes)
            )
    
    def forward(self, x):
        x = self.features(x)    #conv5
        x_ori = x  
        # detect branch
        x = self.downconv(x) # qua 1x1
        x_conv = x              
        x = self.GMP(x)        
        x = self.spatial_pooling(x) 
        x = x.view(x.size(0), -1) #v
        # cls branch
        x_conv = self.spatial_pooling(x_conv) 
        x_conv = x_conv * x.view(x.size(0),x.size(1),1,1) #vi*cimap
        x_conv = self.spatial_pooling2(x_conv)  #M
        x_conv_copy = x_conv  #M
        for num in range(0,511):     #lặp qua mỗi feature trong M       
            x_conv_copy = torch.cat((x_conv_copy, x_conv),1)
        x_conv_copy = torch.mul(x_conv_copy,x_ori)  #U
        #x_conv_copy1= x_conv_copy
        x_conv_copy = torch.cat((x_ori,x_conv_copy),1) 
        #x_conv_copy = torch.cat((x_conv_copy1,x_conv_copy),1) #thêm 
        #x_conv_copy = torch.cat((x_conv_copy,x_ori),1) #thêm 
        x_conv_copy = self.GAP(x_conv_copy)   #d
        x_conv_copy = x_conv_copy.view(x_conv_copy.size(0),-1)
        x_conv_copy = self.classifier(x_conv_copy)
        return x, x_conv_copy

In [6]:
from torch.autograd import Function, Variable
class ClassWisePoolFunction(Function):
    @staticmethod
    def forward(ctx, input, num_maps):
        ctx.num_maps = num_maps
        
        batch_size, num_channels, h, w = input.size()

        if num_channels % ctx.num_maps != 0:
            print('Error in ClassWisePoolFunction. The number of channels has to be a multiple of the number of maps per class')
            sys.exit(-1)

        num_outputs = int(num_channels / ctx.num_maps)
        x = input.view(batch_size, num_outputs, ctx.num_maps, h, w)
        output = torch.sum(x, 2)
        ctx.save_for_backward(input)
        return output.view(batch_size, num_outputs, h, w) / ctx.num_maps

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        
        batch_size, num_channels, h, w = input.size()
        num_outputs = grad_output.size(1)

        grad_input = grad_output.view(batch_size, num_outputs, 1, h, w).expand(batch_size, num_outputs, ctx.num_maps,
                                                                               h, w).contiguous()
        return grad_input.view(batch_size, num_channels, h, w), None

class ClassWisePool(nn.Module):
    def __init__(self, num_maps):
        super(ClassWisePool, self).__init__()
        self.num_maps = num_maps

    def forward(self, input):
        return ClassWisePoolFunction.apply(input, self.num_maps)

In [7]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, num_maps, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0
    
    model_ft = models.resnet18(pretrained=True)
    set_parameter_requires_grad(model_ft, feature_extract)

    pooling = nn.Sequential()
    pooling.add_module('class_wise1', ClassWisePool(num_maps))
    pooling2 = nn.Sequential()
    pooling2.add_module('class_wise2', ClassWisePool(num_classes))
    model_ft = ResNetWSL(model_ft, num_classes, num_maps, pooling, pooling2)
        
    input_size = 448
    
    return model_ft, input_size


model_ft, input_size = initialize_model(model_name, num_classes, num_maps, feature_extract, use_pretrained=True)


'''model_ft.load_state_dict(checkpoint['model_state_dict']) # load pretrained model'''
print(model_ft)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

8
8
ResNetWSL(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [8]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(input_size),
        #transforms.RandomHorizontalFlip(),
        transforms.Resize(input_size),
        #transforms.CenterCrop(input_size),
        #transforms.ColorJitter(brightness=1, contrast=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        #transforms.CenterCrop(input_size),
        #transforms.ColorJitter(brightness=1, contrast=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")


image_datasets = {x: datasets.ImageFolder(os.path.join('../input/fer2013/', x), data_transforms[x]) for x in ['train', 'test']}


dataloaders_dict = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'test']}

class_names = image_datasets['train'].classes
print(class_names)


Initializing Datasets and Dataloaders...
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using GPU ", device)   
model_ft = model_ft.to(device)


params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)


optimizer_ft = optim.SGD(params_to_update, lr=0.0001, momentum=0.9)
'''optimizer_ft.load_state_dict(checkpoint['optimizer_state_dict']) # load pretrained model's optimizer'''
scheduler_ft = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.5)

Using GPU  cuda:0
Params to learn:
	 features.0.weight
	 features.1.weight
	 features.1.bias
	 features.4.0.conv1.weight
	 features.4.0.bn1.weight
	 features.4.0.bn1.bias
	 features.4.0.conv2.weight
	 features.4.0.bn2.weight
	 features.4.0.bn2.bias
	 features.4.1.conv1.weight
	 features.4.1.bn1.weight
	 features.4.1.bn1.bias
	 features.4.1.conv2.weight
	 features.4.1.bn2.weight
	 features.4.1.bn2.bias
	 features.5.0.conv1.weight
	 features.5.0.bn1.weight
	 features.5.0.bn1.bias
	 features.5.0.conv2.weight
	 features.5.0.bn2.weight
	 features.5.0.bn2.bias
	 features.5.0.downsample.0.weight
	 features.5.0.downsample.1.weight
	 features.5.0.downsample.1.bias
	 features.5.1.conv1.weight
	 features.5.1.bn1.weight
	 features.5.1.bn1.bias
	 features.5.1.conv2.weight
	 features.5.1.bn2.weight
	 features.5.1.bn2.bias
	 features.6.0.conv1.weight
	 features.6.0.bn1.weight
	 features.6.0.bn1.bias
	 features.6.0.conv2.weight
	 features.6.0.bn2.weight
	 features.6.0.bn2.bias
	 features.6.0.downsampl

In [10]:

criterion = nn.CrossEntropyLoss()


model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,
                             num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 1/20
----------
train Loss: 1.4219 Acc: 0.4463 Dec: 0.4587
test Loss: 1.1970 Acc: 0.5386 Dec: 0.5518

Epoch 2/20
----------
train Loss: 1.1244 Acc: 0.5754 Dec: 0.5837
test Loss: 1.0697 Acc: 0.6031 Dec: 0.5996

Epoch 3/20
----------
train Loss: 0.9951 Acc: 0.6287 Dec: 0.6281
test Loss: 1.0684 Acc: 0.6095 Dec: 0.6027

Epoch 4/20
----------
train Loss: 0.8992 Acc: 0.6672 Dec: 0.6646
test Loss: 1.0471 Acc: 0.6164 Dec: 0.6192

Epoch 5/20
----------
train Loss: 0.8147 Acc: 0.7017 Dec: 0.6965
test Loss: 1.0571 Acc: 0.6314 Dec: 0.6306

Epoch 6/20
----------
train Loss: 0.5831 Acc: 0.7940 Dec: 0.7787
test Loss: 1.1016 Acc: 0.6328 Dec: 0.6266

Epoch 7/20
----------
train Loss: 0.4815 Acc: 0.8340 Dec: 0.8139
test Loss: 1.1530 Acc: 0.6433 Dec: 0.6353

Epoch 8/20
----------
train Loss: 0.4246 Acc: 0.8575 Dec: 0.8318
test Loss: 1.3393 Acc: 0.6380 Dec: 0.6263

Epoch 9/20
----------
train Loss: 0.3702 Acc: 0.8814 Dec: 0.8536
test Loss: 1.2969 Acc: 0.6330 Dec: 0.6271

Epoch 10/20
----------
train

In [11]:
#torch.save({
 #           'model_state_dict': model_ft.state_dict(),
  #          'optimizer_state_dict': optimizer_ft.state_dict(),
   #         }, 'fres101bat10num8d3.pth')